<a href="https://colab.research.google.com/github/abdulkadirdemirci/CustomerLifetimeValue-CLTV-/blob/main/cltv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

################################
# iş problemi
################################

"""
FLO satışvepazarlamafaaliyetleriiçinroadmap belirlemekistemektedir.
Şirketinortauzunvadeliplan yapabilmesiiçinvar olan müşterilerin
gelecekte şirkete sağlayacakları potansiyel değerin tahmin edilmesi gerekmektedir.
"""

################################
# veri seti hikayesi
################################

"""
Veri seti Flo’dan son alışverişlerini 2020 -2021 yıllarında 
OmniChannel(hem online hem offline alışverişyapan) olarak yapan
müşterilerin geçmiş alışveriş davranışlarından elde edilen bilgilerden oluşmaktadır.

master_id              - Eşsiz müşteri numarası
order_channel          - Alışveriş yapılan platforma ait hangi kanalın kullanıldığı(Android, ios, Desktop, Mobile)
last_order_channel     - En son alışverişin yapıldığı kanal
first_order_date       - Müşterinin yaptığı ilk alışveriş tarihi
last_order_date        - Müşterinin yaptığı son alışveriş tarihi
last_order_date_online        - Müşterinin online platformda yaptığı son alışveriş tarihi
last_order_date_offline       - Müşterinin offline platformda yaptığı son alışveriş tarihi
order_num_total_ever_online   - Müşterinin online platformda yaptığı toplam alışveriş sayısı
order_num_total_ever_offline  - Müşterinin offline'da yaptığı toplam alışveriş sayısı
customer_value_total_ever_offline - Müşterinin offline alışverişlerinde ödediği toplam ücret
customer_value_total_ever_online  - Müşterinin online alışverişlerinde ödediği toplam ücret
interested_in_categories_12       - Müşterinin son 12 ayda alışveriş yaptığı kategorilerin listesi
"""

################################
# GÖREV1 : VERİYİ ANLAMA VE HAZIRLAMA
################################

In [ ]:
!pip install lifetimes

In [4]:
import pandas as pd
import lifetimes
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
pd.set_option("display.max_columns",None)
pd.set_option("display.width",500)

################################
# GÖREV1 - ADIM1 : flo_data_20K.csv verisin iokuyunuz
################################

In [5]:
df_ = pd.read_csv("/content/drive/MyDrive/flo_data_20k.csv")
df = df_.copy()

In [6]:
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,799.38,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,395.35,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,81.98,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,159.99,[AKTIFSPOR]


In [7]:
df.describe((0.01, 0.05, 0.25, 0.5, 0.75, 0.90, 0.99, 1)).T

,count,mean,std,min,1%,5%,25%,50%,75%,90%,99%,100%,max
order_num_total_ever_online,19945.0,3.110855,4.225647,1.00,1.00,1.00,1.00,2.00,4.00,7.000,20.0000,200.00,200.00
order_num_total_ever_offline,19945.0,1.913913,2.062880,1.00,1.00,1.00,1.00,1.00,2.00,4.000,7.0000,109.00,109.00
customer_value_total_ever_offline,19945.0,253.922597,301.532853,10.00,19.99,39.99,99.99,179.98,319.97,519.950,1219.9468,18119.14,18119.14
customer_value_total_ever_online,19945.0,497.321690,832.601886,12.99,39.99,63.99,149.98,286.46,578.44,1082.036,3143.8104,45220.13,45220.13


In [8]:
df.isnull().sum()

master_id                            0
order_channel                        0
last_order_channel                   0
first_order_date                     0
last_order_date                      0
last_order_date_online               0
last_order_date_offline              0
order_num_total_ever_online          0
order_num_total_ever_offline         0
customer_value_total_ever_offline    0
customer_value_total_ever_online     0
interested_in_categories_12          0
dtype: int64

################################
# GÖREV1 - ADIM2 : Aykırı değerleri baskılamak için gerekli olan outlier_thresholds ve
# replace_with_thresholds fonksiyonlarını tanımlayınız.
# Not: cltv hesaplanırken frequency değerleri integer olması gerekmektedir.
# Bu nedenle alt ve üst limitlerini round() ile yuvarlayınız
################################

In [9]:
def outlier_thresholds(df,variablename,upper_quantile,lower_quantile,limiter=1.5):
    """
    outliers(aykırı deger) lara karar verebilmek için boxplot mantığındaki whiskerslar ile belirtilen
    aykırı değerleri değil kendimiz belirlediğimiz sınırı aşan değerleri outlier olarak degerlendirmemizi
    saglayan bir fonksiyon. Temelde quantilelar 0.25 lik ve 0.75 lik dilimlerde bulunur fakat bu fonksiyon
    sayesinde biz istenilen genişlikte quantilelara sahip olabiliriz.

    :param df: üzerinde işlem yapılacak dataframe, type: dataframe
    :param variablename: değişken isimleri, type: str
    :param upper_quantile: quantile3 için yüzdelik kesim, type: int or float
    :param lower_quantile: quantile1 için yüzdelik kesim, type: int or float
    :param limiter: upper ve lower limit için kullanılacak kat sayı, type: int or float, default: 1.5
    :return: up_limit type: int or float, low_limit type: int or float
    """
    quantile1 = df[variablename].quantile(lower_quantile)
    quantile3 = df[variablename].quantile(upper_quantile)
    interquantile_range = quantile3-quantile1
    up_limit  = round(quantile3 + limiter*interquantile_range)
    low_limit = round(quantile1 - limiter*interquantile_range)
    return up_limit,low_limit

In [10]:
def replace_with_thresholds(df,variablename,upper_quantile,lower_quantile,limiter=1.5):
    """

    :param df: üzerinde işlem yapılacak dataframe, type: dataframe
    :param variablename: değişken isimleri, type: str
    :param upper_quantile: quantile3 için yüzdelik kesim, type: int or float
    :param lower_quantile: quantile1 için yüzdelik kesim, type: int or float
    :param limiter: upper ve lower limit için kullanılacak kat sayı, type: int or float, default: 1.5
    :return:
    """
    up_limit, low_limit = outlier_thresholds(df,variablename,upper_quantile,lower_quantile,limiter=1.5)
    df.loc[(df[variablename] > up_limit),  variablename] = up_limit
    df.loc[(df[variablename] < low_limit), variablename] = low_limit

################################
# GÖREV1 - ADIM3 :  "order_num_total_ever_online", "order_num_total_ever_offline",
# "customer_value_total_ever_offline", "customer_value_total_ever_online"
# değişkenlerinin aykırı değerleri varsa baskılayanız
################################

In [11]:
replace_with_thresholds(df, "order_num_total_ever_online", 0.99, 0.01)
replace_with_thresholds(df, "order_num_total_ever_offline", 0.99, 0.01)
replace_with_thresholds(df, "customer_value_total_ever_offline", 0.99, 0.01)
replace_with_thresholds(df, "customer_value_total_ever_online", 0.99, 0.01)

In [12]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.0,3.091953,3.809541,1.00,1.00,2.00,4.00,48.0
order_num_total_ever_offline,19945.0,1.886187,1.434694,1.00,1.00,1.00,2.00,16.0
customer_value_total_ever_offline,19945.0,251.921320,251.023670,10.00,99.99,179.98,319.97,3020.0
customer_value_total_ever_online,19945.0,489.705676,632.609844,12.99,149.98,286.46,578.44,7800.0


################################
# GÖREV1 - ADIM4 : Omnichannel müşterilerin hem online'dan hem de offline
# platformlardan alışveriş yaptığını ifade etmektedir. Her bir müşterinin toplam
# alışveriş sayısı ve harcaması için yeni değişkenler oluşturunuz
################################

In [13]:
df["over_all_order"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["over_all_value"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]

In [15]:
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12,over_all_order,over_all_value
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.0,1.0,139.99,799.38,[KADIN],5.0,939.37
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.0,2.0,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]",21.0,2013.55
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.0,2.0,189.97,395.35,"[ERKEK, KADIN]",5.0,585.32
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.0,1.0,39.99,81.98,"[AKTIFCOCUK, COCUK]",2.0,121.97
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.0,1.0,49.99,159.99,[AKTIFSPOR],2.0,209.98


################################
# GÖREV1 - ADIM5 : Değişken tiplerini inceleyiniz. Tarih ifadeeden değişkenlerin tipini date'e çeviriniz
################################

In [16]:
import datetime as dt

In [17]:
df["first_order_date"]       =pd.to_datetime(df["first_order_date"])
df["last_order_date"]        =pd.to_datetime(df["last_order_date"])
df["last_order_date_online"] =pd.to_datetime(df["last_order_date_online"])
df["last_order_date_offline"]=pd.to_datetime(df["last_order_date_offline"])

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19945 entries, 0 to 19944
Data columns (total 14 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   master_id                          19945 non-null  object        
 1   order_channel                      19945 non-null  object        
 2   last_order_channel                 19945 non-null  object        
 3   first_order_date                   19945 non-null  datetime64[ns]
 4   last_order_date                    19945 non-null  datetime64[ns]
 5   last_order_date_online             19945 non-null  datetime64[ns]
 6   last_order_date_offline            19945 non-null  datetime64[ns]
 7   order_num_total_ever_online        19945 non-null  float64       
 8   order_num_total_ever_offline       19945 non-null  float64       
 9   customer_value_total_ever_offline  19945 non-null  float64       
 10  customer_value_total_ever_online  

################################
# GÖREV2 : CLTV Veri Yapısının Oluşturulması
################################

################################
# GÖREV2 - ADIM1 : Veri setindeki en son alışverişin yapıldığı tarihten 2 gün sonrasını analiz tarihi olarak alınız.
################################

In [19]:
df["last_order_date"].max()
today = dt.datetime(2021, 6, 1)

################################
# GÖREV2 - ADIM2 : customer_id, recency_cltv_weekly, T_weekly, frequency ve monetary_cltv_avg değerlerinin
# yer aldığı yeni bir cltv dataframe'i oluşturunuz. Monetary değeri satın alma başına ortalama değer olarak,
# recency ve tenure değerleri ise haftalık cinsten ifade edilecek
################################

In [20]:
df["recency_cltv_weekly"] = df.apply(lambda row: ((row["last_order_date"] - row["first_order_date"]).days)/7, axis=1)
df["T_weekly"] = df.apply(lambda row: ((today - row["first_order_date"]).days)/7, axis=1)
df["frequency"] = df["over_all_order"]
df["monetary_cltv_avg"]  = df["over_all_value"] / df["frequency"]

In [21]:
cltv = pd.DataFrame({"customer_id":df["master_id"],
                     "recency_cltv_weekly":df["recency_cltv_weekly"],
                     "T_weekly":df["T_weekly"],
                     "frequency":df["frequency"],
                     "monetary_cltv_avg":df["monetary_cltv_avg"]},
                    columns=["customer_id", "recency_cltv_weekly", "T_weekly", "frequency", "monetary_cltv_avg"])


In [22]:
cltv.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.000000,30.571429,5.0,187.874000
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.857143,224.857143,21.0,95.883333
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.285714,78.857143,5.0,117.064000
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.571429,20.857143,2.0,60.985000
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.142857,95.428571,2.0,104.990000


################################
# GÖREV3 : BG/NBD, Gamma-Gamma Modellerinin Kurulması ve CLTV’ninHesaplanması
################################

################################
# GÖREV3 - ADIM1 :  BG/NBD modelinifit ediniz.
#   a. 3 ay içerisinde müşterilerden beklenen satın almaları tahmin ediniz
#   ve exp_sales_3_month olarak cltv dataframe'ine ekleyiniz.
#   b. 6 ay içerisinde müşterilerden beklenen satın almaları tahmin ediniz
#   ve exp_sales_6_month olarak cltv dataframe'ine ekleyiniz
################################

In [23]:
cltv.describe().T

,count,mean,std,min,25%,50%,75%,max
recency_cltv_weekly,19945.0,95.263496,74.589446,0.000000,50.428571,76.571429,109.428571,433.428571
T_weekly,19945.0,114.471833,74.770983,0.714286,73.857143,93.000000,119.428571,437.142857
frequency,19945.0,4.978140,4.120919,2.000000,3.000000,4.000000,6.000000,57.000000
monetary_cltv_avg,19945.0,152.099009,73.524091,22.490000,103.653333,136.906154,182.450000,1401.800000


In [24]:
bgf = BetaGeoFitter(penalizer_coef=0.001)

In [25]:
bgf.fit(cltv["frequency"],
        cltv["recency_cltv_weekly"],
        cltv["T_weekly"])

<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 76.17, b: 0.00, r: 3.66>

In [26]:
bgf.conditional_expected_number_of_purchases_up_to_time(3*4,
                                                        cltv["frequency"],
                                                        cltv["recency_cltv_weekly"],
                                                        cltv["T_weekly"]).sort_values(ascending=False).head(10)


7330     4.656138
15611    3.373958
8328     3.142396
19538    3.083779
14373    3.001287
10489    2.978047
4315     2.829904
6756     2.793429
6666     2.780689
10536    2.763492
dtype: float64

In [27]:
cltv["exp_sales_3_month"] = bgf.conditional_expected_number_of_purchases_up_to_time(3*4,
                                                        cltv["frequency"],
                                                        cltv["recency_cltv_weekly"],
                                                        cltv["T_weekly"])


In [28]:
cltv["exp_sales_6_month"] = bgf.conditional_expected_number_of_purchases_up_to_time(6*4,
                                                        cltv["frequency"],
                                                        cltv["recency_cltv_weekly"],
                                                        cltv["T_weekly"])


In [29]:
cltv.head(10)

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.000000,30.571429,5.0,187.874000,0.973927,1.947853
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.857143,224.857143,21.0,95.883333,0.983161,1.966323
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.285714,78.857143,5.0,117.064000,0.670586,1.341172
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.571429,20.857143,2.0,60.985000,0.700412,1.400824
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.142857,95.428571,2.0,104.990000,0.396039,0.792077
5,e585280e-aae1-11e9-a2fc-000d3a38a36f,120.857143,132.285714,3.0,66.953333,0.383581,0.767162
6,c445e4ee-6242-11ea-9d1a-000d3a38a36f,32.571429,64.857143,4.0,93.982500,0.652066,1.304133
7,3f1b4dc8-8a7d-11ea-8ec0-000d3a38a36f,12.714286,54.571429,2.0,81.815000,0.519800,1.039600
8,cfbda69e-5b4f-11ea-aca7-000d3a38a36f,58.428571,70.714286,5.0,210.938000,0.707761,1.415522
9,1143f032-440d-11ea-8b43-000d3a38a36f,61.714286,96.000000,2.0,82.980000,0.394724,0.789448


################################
# GÖREV3 - ADIM2 :  Gamma-Gamma modelini fit ediniz.
# Müşterilerin ortalama bırakacakları değeri tahminleyip
# exp_average_value olarak cltv dataframe'ine ekleyiniz
################################

In [30]:
ggf = GammaGammaFitter(penalizer_coef=0.001)

In [31]:
ggf.fit(cltv["frequency"],cltv["monetary_cltv_avg"])


<lifetimes.GammaGammaFitter: fitted with 19945 subjects, p: 13.19, q: 1.70, v: 12.96>

In [32]:
ggf.conditional_expected_average_profit(cltv["frequency"],cltv["monetary_cltv_avg"]).sort_values(ascending=False).head(10)


9055     1386.685885
17323    1099.998782
15516    1079.314143
6402      846.754436
16410     843.724638
7936      833.948942
9738      826.785361
1853      820.630957
2291      771.265155
12828     765.480850
dtype: float64

In [33]:
cltv.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.000000,30.571429,5.0,187.874000,0.973927,1.947853
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.857143,224.857143,21.0,95.883333,0.983161,1.966323
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.285714,78.857143,5.0,117.064000,0.670586,1.341172
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.571429,20.857143,2.0,60.985000,0.700412,1.400824
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.142857,95.428571,2.0,104.990000,0.396039,0.792077


################################
# GÖREV3 - ADIM3 :  6 aylık CLTV hesaplayınız ve cltv ismiyle dataframe'e ekleyiniz.
#   a. Cltv değeri en yüksek 20 kişiyi gözlemleyiniz.
################################

In [34]:
cltv["cltv_6_month"] = ggf.customer_lifetime_value(bgf,
                                                   cltv["frequency"],
                                                   cltv["recency_cltv_weekly"],
                                                   cltv["T_weekly"],
                                                   cltv["monetary_cltv_avg"],
                                                   time=6,
                                                   freq = "W",
                                                   discount_rate=0.01)

In [35]:
cltv.sort_values(by="cltv_6_month",ascending=False).head(10)

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,cltv_6_month
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,2.857143,7.857143,4.0,1401.800000,1.094385,2.188769,3184.533393
13880,7137a5c0-7aad-11ea-8f20-000d3a38a36f,6.142857,13.142857,11.0,758.085455,1.970108,3.940216,3123.884407
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,51.714286,101.000000,7.0,1106.467143,0.722238,1.444476,1667.132825
12438,625f40a2-5bd2-11ea-98b0-000d3a38a36f,74.285714,74.571429,16.0,501.873750,1.565309,3.130618,1645.727174
7330,a4d534a2-5b1b-11eb-8dbd-000d3a38a36f,62.714286,67.285714,52.0,166.224615,4.656138,9.312276,1624.903296
8868,9ce6e520-89b0-11ea-a6e7-000d3a38a36f,3.428571,34.428571,8.0,601.226250,1.265456,2.530912,1590.327250
6666,53fe00d4-7b7a-11eb-960b-000d3a38a36f,9.714286,13.000000,17.0,259.865294,2.780689,5.561378,1516.072650
19538,55d54d9e-8ac7-11ea-8ec0-000d3a38a36f,52.571429,58.714286,31.0,228.530000,3.083779,6.167558,1479.029144
6402,851de3b4-8f0c-11eb-8cb8-000d3a38a36f,8.285714,9.428571,2.0,862.690000,0.793924,1.587847,1410.697550
4157,7eed6468-4540-11ea-acaf-000d3a38a36f,89.142857,90.000000,27.0,289.764074,2.214331,4.428662,1346.023793


################################
# GÖREV4 : CLTV Değerine Göre Segmentlerin Oluşturulması
################################

################################
# GÖREV4 - ADIM1 :   6 aylık CLTV'ye göre tüm müşterilerinizi 4 gruba(segmente)
# ayırınız ve grup isimlerini verisetine ekleyiniz.
################################

In [36]:
cltv["segment"] = pd.qcut(cltv["cltv_6_month"],4,labels=["D","C","B","A"])

In [37]:
cltv.segment.value_counts()

D    4987
C    4986
B    4986
A    4986
Name: segment, dtype: int64

################################
# GÖREV4 - ADIM2 :   4 grup içerisinden seçeceğiniz 2 grup için yönetime kısa kısa
# 6 aylık aksiyon önerilerinde bulununuz.
################################

In [38]:
cltv.groupby("segment").agg({"cltv_6_month":["min","max","std","mean","count"],
                            "T_weekly":["min","max","std","mean","count"],
                            "monetary_cltv_avg":["min","max","std","mean","count"],
                            "frequency":["min","max","std","mean","count"]})


cltv_6_month                                             T_weekly                                          monetary_cltv_avg                                           frequency                                
                 min          max         std        mean count       min         max        std        mean count               min          max        std        mean count       min   max       std      mean count
segment                                                                                                                                                                                                                 
D          13.042048   108.639926   20.850167   78.088592  4987  3.571429  436.285714  95.723581  161.514710  4987         22.490000   320.100000  31.232911   93.746344  4987       2.0  18.0  2.107212  3.703830  4987
C         108.640087   159.775086   14.743029  133.702977  4986  2.857143  433.428571  63.070405  112.271761  4986         34.315556   366.996667  32.685699  126.567406  4986       2.0  26.0  2.790881  4.324108  4986
B         159.777598   231.426512   20.447041  192.659635  4986  1.142857  434.714286  55.740568  100.124205  4986         50.042632   539.985000  41.584370  160.549088  4986       2.0  40.0  3.585314  5.076815  4986
A         231.446369  3184.533393  154.340471  350.228857  4986  0.714286  437.142857  52.556463   83.967223  4986         64.364500  1401.800000  89.616767  227.544900  4986       2.0  57.0  6.119265  6.808063  4986